In [1]:
%pylab inline
import numpy as np
import pandas as pd
data = []
datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_2014.01.23.csv'
data = pd.read_csv(datapath)
#data_CNF14 = data[data.Series == 'CNF14']
#data = data_CNF14
data.head()

Populating the interactive namespace from numpy and matplotlib


,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity
0,CNF14,1,2014-01-23D04:30:05.479815300,E1C9F25394A679A6,1,719500,1,,A,688500,20
1,CNF14,2,2014-01-23D04:30:05.480815300,C791DA0F799A5E9A,1,710000,2,,A,688500,20
2,CNF14,3,2014-01-23D04:30:05.480815300,614220B5BE75A2B5,1,685500,5,,B,686500,10
3,CNF14,4,2014-01-23D04:30:05.480815300,6693A4751AF1FBF2,1,689000,20,,A,688500,20
4,CNF14,5,2014-01-23D04:30:05.480815300,E6F0E7100CF1EF52,2,676000,15,,B,686500,10


In [2]:
data_CNF = data[data.Series == data.Series[0]]
data = data_CNF

In [8]:
len(data)

115354

In [3]:
import time 
def insert(order_book_data,data_to_insert,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position:]
    return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)

start  =  time.time()

order_book_bid = []
order_book_ask = []
timestamp = data.TimeStamp[0]
temp_ask = 0
temp_bid = 0
bid = []
ask = []
timestamp_ = []
x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
y = data[data['TimeStamp'].str.contains(timestamp)]
bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference"]]) # bid
ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference"]]) # ask
a = bid[0].sort(['Price'],ascending = [False])
b = ask[0].sort(['Price'],ascending = [True])    
order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))

order_book_bid.append(0)
order_book_ask.append(0)
#a = []
#b = []

for i in range(len(y),3200,1):
    print '------------------------------------------------------------------------------'
    print i
    #print data.BidOrAsk[i] == 'A'
    if data.BidOrAsk[i] == 'A':

        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            
            if data.TimeStamp[i] == x1[temp_ask]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)

                if position_ == 0 and len(order_book_ask[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:#or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error2 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'   
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:# or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error3 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                
                        continue
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True' 
                
            elif data.TimeStamp[i] != x1[temp_ask]:
                ###a.append(x1[temp_ask])
                #print order_book_ask[temp_ask]
                #print temp_ask
                if temp_ask == 0:
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                    #print order_book_ask[temp_ask]
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    #print order_book_ask[temp_ask]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    #print 'position = %d'%(position_)
                    #print 'length-1 = %d'%(len(order_book_ask[temp_ask])-1)
                    #print order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                    #print order_book_ask[temp_ask][position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                    #print order_book_ask[temp_ask]['Price'][0:1].iloc[0]# != data['BestPrice'][i]
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error11 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                    
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error22 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        
                        if order_book_ask[temp_ask][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:# or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error33 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            continue
                        else:
                            print 'position and Best Price is True'
                            continue
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        continue
                        
                    #print order_book_ask[temp_ask]                        
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                    order_book_ask.append(order_book_ask[temp_ask])
                    temp_ask = temp_ask + 1

                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)

                        
                    if position_ == 0:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                        
                            continue
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        continue                            
                            
                            
                            
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x1[temp_ask]: 
                #print temp_ask
                #print order_book_ask[temp_ask]
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_ask[temp_ask]

                if position_ == 0 and len(order_book_ask[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error1111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error2222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error3333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue                        
                    else:
                        print 'position and Best Price is True'                        
                        continue
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True'
                    continue

            elif data.TimeStamp[i] != x1[temp_ask]:
                ###a.append(x1[temp_ask])
                #print temp_ask
                #print order_book_ask[temp_ask]
                order_book_ask.append(order_book_ask[temp_ask])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_ask = temp_ask + 1
                #print 'temp_ask2 = %d'%(temp_ask)
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_ask[temp_ask]
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error11111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error22222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error33333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue                        
                    else:
                        print 'position and Best Price is True'                        
                        continue 
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True'
                    continue                          

    elif data.BidOrAsk[i] == 'B':
        #print i
        #print int(data[['QuantityDifference']][i:i+1].values) > 0
        
        if int(data[['QuantityDifference']][i:i+1].values) > 0: #and len(order_book_bid[0]) > 0:
            print data.TimeStamp[i] == x2[temp_bid]
            
            if data.TimeStamp[i] == x2[temp_bid]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                #print position_
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error111111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                        
                elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error222222 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'   
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[0]) > 1:
                        
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:# or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error333333 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                
                        continue
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    continue
                
            elif data.TimeStamp[i] != x2[temp_bid]:
                ###b.append(x2[temp_bid])
                #print 'temp bid = %d'%(temp_bid)
                #print order_book_bid[temp_bid]
                if temp_bid == 0:
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                    #print order_book_bid[temp_bid]
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    #print order_book_bid[temp_bid]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    #print position_
                    #print len(order_book_bid[temp_bid]) - 1
                    
                    #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                    #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                    #print order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]
                    #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0]
                    #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0]
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    if position_ == 0  and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1111111 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                            
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2222222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3333333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            continue
                        else:
                            print 'position and Best Price is True'
                            continue
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                            print 'position and Best Price is True'
                            continue
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    order_book_bid.append(order_book_bid[temp_bid])
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    #print position_    
                    #print len(order_book_bid[temp_bid])-1      
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error11111111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error22222222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error33333333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                        
                            continue
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        continue
                        
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x2[temp_bid]: 
                #print temp_bid
                #print order_book_bid[temp_bid]
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_bid[temp_bid]

                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:# or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error19 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                        
                elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error29 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1:
                        
                    if position_ > 0 and order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error39 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue
                    else:
                        print 'position and Best Price is True'                        
                        continue
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    continue

            elif data.TimeStamp[i] != x2[temp_bid]:
                ###b.append(x2[temp_bid])
                #print temp_bid
                #print order_book_bid[temp_bid]
                order_book_bid.append(order_book_bid[temp_bid])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_bid = temp_bid + 1
                #print position_
                #print len(order_book_ask[temp_ask])-1
                #print 'temp_bid2 = %d'%(temp_bid)
                
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_bid[temp_bid]
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error110 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
    
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error220 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1:
                        
                    if position_ > 0 and order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error330 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue
                    else:
                        print 'position and Best Price is True'                        
                        continue    
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    continue                    
                                    
end = time.time()            
print "Total time = %f"%(end - start)    

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


------------------------------------------------------------------------------
1
position and Best Price is True
------------------------------------------------------------------------------
2
True
position and Best Price is True
------------------------------------------------------------------------------
3
position and Best Price is True
------------------------------------------------------------------------------
4
True
position and Best Price is True
------------------------------------------------------------------------------
5
True
position and Best Price is True
------------------------------------------------------------------------------
6
True
position and Best Price is True
------------------------------------------------------------------------------
7
position and Best Price is True
------------------------------------------------------------------------------
8
position and Best Price is True
----------------------------------------------------------------------------

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


position and Best Price is True
------------------------------------------------------------------------------
318
position and Best Price is True
------------------------------------------------------------------------------
319
position and Best Price is True
------------------------------------------------------------------------------
320
position and Best Price is True
------------------------------------------------------------------------------
321
position and Best Price is True
------------------------------------------------------------------------------
322
False
position and Best Price is True
------------------------------------------------------------------------------
323
position and Best Price is True
------------------------------------------------------------------------------
324
False
position and Best Price is True
------------------------------------------------------------------------------
325
True
position and Best Price is True
-------------------------------

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


position and Best Price is True
------------------------------------------------------------------------------
346
position and Best Price is True
------------------------------------------------------------------------------
347
position and Best Price is True
------------------------------------------------------------------------------
348
position and Best Price is True
------------------------------------------------------------------------------
349
position and Best Price is True
------------------------------------------------------------------------------
350
position and Best Price is True
------------------------------------------------------------------------------
351
position and Best Price is True
------------------------------------------------------------------------------
352
position and Best Price is True
------------------------------------------------------------------------------
353
position and Best Price is True
------------------------------------------------

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:429: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


position and Best Price is True
------------------------------------------------------------------------------
395
False
position and Best Price is True
------------------------------------------------------------------------------
396
position and Best Price is True
------------------------------------------------------------------------------
397
position and Best Price is True
------------------------------------------------------------------------------
398
position and Best Price is True
------------------------------------------------------------------------------
399
False
position and Best Price is True
------------------------------------------------------------------------------
400
position and Best Price is True
------------------------------------------------------------------------------
401
position and Best Price is True
------------------------------------------------------------------------------
402
position and Best Price is True
------------------------------------

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:380: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


position and Best Price is True
------------------------------------------------------------------------------
668
position and Best Price is True
------------------------------------------------------------------------------
669
position and Best Price is True
------------------------------------------------------------------------------
670
position and Best Price is True
------------------------------------------------------------------------------
671
position and Best Price is True
------------------------------------------------------------------------------
672
position and Best Price is True
------------------------------------------------------------------------------
673
position and Best Price is True
------------------------------------------------------------------------------
674
position and Best Price is True
------------------------------------------------------------------------------
675
False
position and Best Price is True
------------------------------------------

In [4]:
len(order_book_bid),len(data[0:3200][(data[0:3200].BidOrAsk == 'B')].TimeStamp.unique())

(1010, 1010)

In [5]:
len(order_book_ask),len(data[0:3200][(data[0:3200].BidOrAsk == 'A')].TimeStamp.unique())

(1151, 1151)

In [6]:
order_book_ask[len(order_book_ask)-1]

,Price,OrderNumber,QuantityDifference
0,686500,6903ACB1A0507FB0,10
1,686500,49CAACB9E465C405,10
2,686500,4CE5E350FD54E254,15
3,686500,62942D299C9F7D1F,10
4,686500,4802813DDD71BE11,10
5,686500,CA0A096DD59FB63F,10
6,686500,C41A01257ABF5B5F,9
7,686500,629E812D632C43CC,7
8,686500,6F8924F9D654B5F4,10
9,686500,C94008FDB8CA988A,10


In [7]:
order_book_bid[len(order_book_bid)-1]

,Price,OrderNumber,QuantityDifference
0,685500,4D44A8F9525D315D,20
1,685500,E431EF40440B268B,1
2,685500,C3CEACE964844424,9
3,685500,C6BCEF5037FF1CBF,1
4,685500,685E8D65415021F0,10
5,685500,440A8D2D732B53CB,10
6,685500,428A0965741C54BC,10
7,685500,EE5D8D757EC85F68,10
8,685500,6CD08129243F053F,10
9,685500,4B0A20F9522935E9,10


In [9]:
order_book_bid[0]

,Price,OrderNumber,QuantityDifference
0,686500,CBD480E5F20BD5AB,10
1,685500,614220B5BE75A2B5,5
2,685000,43100CB5A3DF851F,20
3,684500,44B56B5089756BD5,1
4,676000,E6F0E7100CF1EF52,15


In [10]:
order_book_ask[0]

,Price,OrderNumber,QuantityDifference
0,719500,E1C9F25394A679A6,1


In [30]:
a = 300

if '08:5' in data[a:a+1].TimeStamp.iloc[0][11:16] or '08:4' in data[a:a+1].TimeStamp.iloc[0][11:16]:
    print 1 #.contain('08:5')

1


In [31]:
data[a:a+1].TimeStamp.iloc[0][11:16]

'08:59'